In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import csv

In [ ]:
# Load and preprocess the team statistics data
team_season_data = []

with open('/content/drive/MyDrive/databasebasketball(1)/team_season.txt', 'r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        team_season_data.append(row)

data = pd.DataFrame(team_season_data)

In [ ]:
# Define columns to keep for feature selection
columns_to_keep = ['o_fgm', 'o_fga', 'o_ftm', 'o_fta', 'o_oreb', 'o_dreb', 'o_reb', 'o_asts', 'o_pf', 'o_stl', 'o_to', 'o_blk', 'o_3pm', 'o_3pa', 'o_pts',
                   'd_fgm', 'd_fga', 'd_ftm', 'd_fta', 'd_oreb', 'd_dreb', 'd_reb', 'd_asts', 'd_pf', 'd_stl', 'd_to', 'd_blk', 'd_3pm', 'd_3pa', 'd_pts', 'pace']

# Extract features (X) and labels (Y)
X = data[columns_to_keep]
Y = data[['won', 'lost']]  # Keep both 'won' and 'lost' columns for comparison

# Create a new target variable indicating which team won (1 if team1 won, 0 if team2 won)
Y['team1_won'] = np.where(Y['won'] > Y['lost'], 1, 0)

# Define features (X) and labels (Y)
X = X.drop(['pace'], axis=1)  # Remove 'pace' as it is not used in your features
Y = Y['team1_won']

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

<ipython-input-13-7b74c43b42fb>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['team1_won'] = np.where(Y['won'] > Y['lost'], 1, 0)


In [ ]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize and train the SVM model
model = SVC(probability=True, kernel='linear', random_state=42)
model.fit(X_train, Y_train)

# Evaluate the model on the test set
Y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(Y_test, Y_pred))
print("Classification Report:\n", classification_report(Y_test, Y_pred))


Accuracy: 0.9117647058823529
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.90      0.91       120
           1       0.90      0.92      0.91       118

    accuracy                           0.91       238
   macro avg       0.91      0.91      0.91       238
weighted avg       0.91      0.91      0.91       238



In [ ]:
# Define a function to prepare team features for prediction
def prepare_features_for_team(team_name, data):
    # Filter the DataFrame for the specified team
    team_data = data[data['team'] == team_name]
    # Extract relevant features
    features = team_data[['o_fgm', 'o_fga', 'o_ftm', 'o_fta', 'o_oreb', 'o_dreb', 'o_reb', 'o_asts', 'o_pf', 'o_stl', 'o_to', 'o_blk', 'o_3pm', 'o_3pa', 'o_pts',
                          'd_fgm', 'd_fga', 'd_ftm', 'd_fta', 'd_oreb', 'd_dreb', 'd_reb', 'd_asts', 'd_pf', 'd_stl', 'd_to', 'd_blk', 'd_3pm', 'd_3pa', 'd_pts', 'pace']]
    features = features.drop(['pace'], axis=1)
    features = scaler.transform(features)
    return features


In [ ]:
# Get team names and prepare features for prediction
team1_name = input("Enter the name of Team 1: ")
team2_name = input("Enter the name of Team 2: ")

team1_features = prepare_features_for_team(team1_name, data)
team2_features = prepare_features_for_team(team2_name, data)

# Predict win probabilities for each team
team1_win_probability = model.predict_proba(team1_features)[:, 1].mean()  # Probability that team1 wins
team2_win_probability = model.predict_proba(team2_features)[:, 1].mean()  # Probability that team2 wins

# Compare the win probabilities
if team1_win_probability > team2_win_probability:
    print(f"{team1_name} is more likely to win with a probability of {team1_win_probability:.2f}")
else:
    print(f"{team2_name} is more likely to win with a probability of {team2_win_probability:.2f}")

Enter the name of Team 1: PRO
Enter the name of Team 2: INJ
PRO is more likely to win with a probability of 0.12
